## Automating Detection of Non-Functioning Readers

 

In [217]:
import matplotlib.pyplot as plt
import pandas as pd
import configparser
from psycopg2 import connect
import psycopg2.sql as pg
import pandas.io.sql as pandasql
import io
import os


CONFIG = configparser.ConfigParser()
CONFIG.read(r'C:\Users\alouis2\Documents\Python Scripts\db.cfg')
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [144]:
string = ''' SELECT segments.analysis_id
               FROM bluetooth.segments
            EXCEPT
             SELECT DISTINCT f.analysis_id
               FROM (SELECT observations.analysis_id,
                        observations.measured_timestamp
                       FROM bluetooth.observations
                      WHERE observations.measured_timestamp <= (( SELECT max(observations_1.measured_timestamp) AS max
                               FROM bluetooth.observations observations_1)) AND observations.measured_timestamp >= (( SELECT max(observations_1.measured_timestamp) - '02:00:00'::interval
                               FROM bluetooth.observations observations_1)) AND (observations.analysis_id IN ( SELECT segments.analysis_id
                               FROM bluetooth.segments))
                      ORDER BY observations.measured_timestamp DESC) f'''

df = pandasql.read_sql(pg.SQL(string), con)


In [145]:
badroutes = []
for i in range(len(df['analysis_id'])):
    string = '''SELECT analysis_id, startpoint_name, endpoint_name,  measured_timestamp as last_active FROM bluetooth.observations
                WHERE analysis_id = %d 
                ORDER BY measured_timestamp desc 
                LIMIT 1''' % list(df['analysis_id'])[i]
    row = pandasql.read_sql(pg.SQL(string), con)
    badroutes.append(list(row.loc[0]))

badroutes

[[1455710, 'QU_BV', 'EA_BV', Timestamp('2018-02-15 08:26:45')],
 [1455724, 'EA_BV', 'QU_BV', Timestamp('2018-02-15 08:13:30')],
 [1432978, 'BR3', 'BR4', Timestamp('2017-07-21 08:20:11')],
 [1454853, 'QU_RO', 'DU_RO', Timestamp('2017-12-17 17:57:21')],
 [1453680, 'QU_BV', 'QU_PA', Timestamp('2018-02-15 08:18:06')],
 [1454832, 'DU_RO', 'QU_RO', Timestamp('2017-12-17 18:44:11')],
 [1454311, 'QU_BV', 'KN_PA', Timestamp('2018-02-15 08:30:11')],
 [1453667, 'QU_PA', 'QU_BV', Timestamp('2018-02-15 08:01:21')],
 [1412684, 'AD2', 'AD1', Timestamp('2017-12-17 18:00:14')],
 [1412641, 'AD1', 'AD2', Timestamp('2017-12-17 18:47:53')],
 [1432982, 'BR2', 'BR3', Timestamp('2017-07-21 09:13:37')],
 [1453239, 'DU_RO', 'DU_DF', Timestamp('2017-12-17 18:51:17')],
 [1454293, 'KN_PA', 'QU_BV', Timestamp('2018-02-15 08:13:30')],
 [1453507, 'DU_DF', 'DU_RO', Timestamp('2017-12-17 17:59:21')]]

In [146]:
badroutes = pd.DataFrame(badroutes, columns = list(row.columns.values))

In [175]:
badroutes = badroutes.sort_values(by='last_active').reset_index(drop=True)
badroutes

,analysis_id,startpoint_name,endpoint_name,last_active
0,1432978,BR3,BR4,2017-07-21 08:20:11
1,1432982,BR2,BR3,2017-07-21 09:13:37
2,1454853,QU_RO,DU_RO,2017-12-17 17:57:21
3,1453507,DU_DF,DU_RO,2017-12-17 17:59:21
4,1412684,AD2,AD1,2017-12-17 18:00:14
5,1454832,DU_RO,QU_RO,2017-12-17 18:44:11
6,1412641,AD1,AD2,2017-12-17 18:47:53
7,1453239,DU_RO,DU_DF,2017-12-17 18:51:17
8,1453667,QU_PA,QU_BV,2018-02-15 08:01:21
9,1455724,EA_BV,QU_BV,2018-02-15 08:13:30


In [149]:
string2 = '''SELECT analysis_id, startpoint_name, endpoint_name
            FROM bluetooth.observations
                WHERE observations.measured_timestamp <= (( SELECT max(observations_1.measured_timestamp) AS max
                FROM bluetooth.observations observations_1)) AND observations.measured_timestamp >= (( SELECT max(observations_1.measured_timestamp) - '02:00:00'::interval
                FROM bluetooth.observations observations_1)) AND (observations.analysis_id IN ( SELECT segments.analysis_id
                FROM bluetooth.segments))
                GROUP BY analysis_id, startpoint_name, endpoint_name;'''

df2 = pandasql.read_sql(pg.SQL(string2), con)

In [150]:
df2.head(5)

,analysis_id,startpoint_name,endpoint_name
0,116449,J,I
1,116451,I,J
2,116749,A,B
3,116796,F,G
4,116804,G,H


In [206]:
final = []
for i in range(len(badroutes)): 
    if (badroutes['startpoint_name'].values[i] not in df2['startpoint_name'].values \
    and badroutes['startpoint_name'].values[i] not in df2['endpoint_name'].values):
        final.append(badroutes['startpoint_name'].values[i])
for i in range(len(badroutes)):
    if badroutes['endpoint_name'].values[i] not in df2['startpoint_name'].values \
    and badroutes['endpoint_name'].values[i] not in df2 ['endpoint_name'].values:
        final.append(badroutes['endpoint_name'].values[i])
        
final = list(set(final))

for i in final: 
    j = (len(badroutes))-1
    while j < len(badroutes):
        if i == badroutes['startpoint_name'].values[j]:
            final[final.index(i)] = ([i, badroutes['last_active'].values[j]])
            break
        elif i == badroutes['endpoint_name'].values[j]:
            final[final.index(i)] = ([i, badroutes['last_active'].values[j]])
            break
        else:
            j = j-1

for i in final:
    final[final.index(i)].append([])
    starts = list(badroutes['startpoint_name'].values)
    ends = list(badroutes['endpoint_name'].values)
    j = 0 
    while j < len(starts):
        if i[0] == starts[j]: 
            final[final.index(i)][2].append('' + starts[j] + ' to ' + ends[j] + '')
            j += 1
        else:
            j += 1
    j = 0
    while j < len(ends):
        if i[0] == ends[j]:
            final[final.index(i)][2].append('' + starts[j] + ' to ' + ends[j] + '')
            j += 1
        else:
            j += 1
    
for i in range(len(badroutes)): 
    if (badroutes['startpoint_name'].values[i] in df2['startpoint_name'].values or
        badroutes['startpoint_name'].values[i] in df2['endpoint_name'].values)\
    and (badroutes['endpoint_name'].values[i] in df2['startpoint_name'].values or
         badroutes['endpoint_name'].values[i] in df2['endpoint_name'].values):
        final.append(['-', '-', badroutes['startpoint_name'].values[i]])
        
final

[['QU_BV',
  numpy.datetime64('2018-02-15T08:30:11.000000000'),
  ['QU_BV to QU_PA',
   'QU_BV to EA_BV',
   'QU_BV to KN_PA',
   'QU_PA to QU_BV',
   'EA_BV to QU_BV',
   'KN_PA to QU_BV']],
 ['DU_RO',
  numpy.datetime64('2017-12-17T18:51:17.000000000'),
  ['DU_RO to QU_RO', 'DU_RO to DU_DF', 'QU_RO to DU_RO', 'DU_DF to DU_RO']],
 ['BR4', numpy.datetime64('2017-07-21T08:20:11.000000000'), ['BR3 to BR4']],
 ['BR3',
  numpy.datetime64('2017-07-21T09:13:37.000000000'),
  ['BR3 to BR4', 'BR2 to BR3']],
 ['AD1',
  numpy.datetime64('2017-12-17T18:47:53.000000000'),
  ['AD1 to AD2', 'AD2 to AD1']]]

In [212]:
pd.set_option("max_columns",10)
broken_readers = pd.DataFrame(final, columns = ['Reader', 'Last Active', 'Routes Affected'])
broken_readers

,Reader,Last Active,Routes Affected
0,QU_BV,2018-02-15 08:30:11,"[QU_BV to QU_PA, QU_BV to EA_BV, QU_BV to KN_P..."
1,DU_RO,2017-12-17 18:51:17,"[DU_RO to QU_RO, DU_RO to DU_DF, QU_RO to DU_R..."
2,BR4,2017-07-21 08:20:11,[BR3 to BR4]
3,BR3,2017-07-21 09:13:37,"[BR3 to BR4, BR2 to BR3]"
4,AD1,2017-12-17 18:47:53,"[AD1 to AD2, AD2 to AD1]"


In [216]:
path="C:\\Users\\alouis2\\Documents"
broken_readers.to_csv(os.path.join(path,r'broken_readers.csv'))